In [ ]:
!pip install pdfquery
!pip install openai
!pip install pdfminer

In [2]:
import pdfplumber
import re

def extract_text_excluding_tables(pdf_path):
    text_without_tables = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            full_text = page.extract_text()
            if full_text is None:
                continue  # Skip pages without text

            text_lines = full_text.split('\n')
            tables = page.find_tables()  # Find tables on the page
            
            # Collect all bounding boxes of tables
            table_bboxes = [table.bbox for table in tables]

            # Process each text line and exclude those that are within table bounds or match table-like patterns
            for line in text_lines:
                exclude = False
                words_in_line = page.extract_words(keep_blank_chars=True, extra_attrs=["doctop", "x0", "x1"])

                # Check if line looks like a table using pattern matching (high count of numbers, $, %, or specific formatting)
                if re.search(r'(\$\s*\d+|%\s*\d+|(\d{1,3},)*\d{1,3}(\.\d{2})?)', line):
                    exclude = True

                # Check if the line is inside any table's bounding box
                if not exclude:
                    for word in words_in_line:
                        for bbox in table_bboxes:
                            if bbox[0] <= word['x0'] <= bbox[2] and bbox[1] <= word['doctop'] <= bbox[3]:
                                exclude = True
                                break
                        if exclude:
                            break
                
                if not exclude:
                    text_without_tables.append(line)

    return text_without_tables





In [3]:
# Extract text from a PDF file excluding tables
pdf_path = "local path to pdf file"
text_data = extract_text_excluding_tables(pdf_path)
text = ""
for line in text_data:
    text += line
    

In [ ]:
print(text)

In [5]:
from openai._client import OpenAI
client = OpenAI(api_key = 'openai api key')

In [6]:
# rewrite function with variable for both content varaibles
def question_answering(transcription, question = 'please summarize the text'):
    response = client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages=[

            {
                "role": "system",
                "content": question
            },
            {
                "role": "user",
                "content": transcription 
            }
        ]
    )
    return response.choices[0].message.content

In [ ]:
question_answering(text)